In [1]:
import pandas as pd
import numpy as np
from os import path
import sys
import tensorflow as tf

#plotting
from matplotlib import pyplot as plt

#processing
from scipy.signal import resample


/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.12) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [15]:
!pip install pandas_profiling

     |████████████████████████████████| 262 kB 1.7 MB/s eta 0:00:01
     |████████████████████████████████| 62 kB 336 kB/s eta 0:00:01
     |████████████████████████████████| 12.8 MB 4.0 MB/s eta 0:00:01
     |████████████████████████████████| 696 kB 4.0 MB/s eta 0:00:01
     |████████████████████████████████| 78 kB 761 kB/s eta 0:00:011
     |████████████████████████████████| 102 kB 2.4 MB/s ta 0:00:01
     |████████████████████████████████| 4.7 MB 1.5 MB/s eta 0:00:01
     |████████████████████████████████| 2.0 MB 2.5 MB/s eta 0:00:01
     |████████████████████████████████| 812 kB 4.7 MB/s eta 0:00:01
     |████████████████████████████████| 6.9 MB 3.7 MB/s eta 0:00:01
  Created wheel for htmlmin: filename=htmlmin-0.1.12-py3-none-any.whl size=27080 sha256=8ce177763ca8b968d2b1fe33896ced65c4d0746fbe914cf3d53bc0ebf56fdc34
  Stored in directory: /root/.cache/pip/wheels/23/14/6e/4be5bfeeb027f4939a01764b48edd5996acf574b0913fe5243
  Created wheel for imagehash: filename=ImageHash-4.2.1-py2.p

In [16]:
import pandas_profiling as pdpf


In [3]:
features=['x','y','pen_up','pressure']
cols=[0, 1, 3, 6]
doc_path = "/data/elekin/doc"
meta_df=pd.read_csv(path.join(doc_path,"metadata-202106-v1.csv"),index_col=0)
x_train = []
y_train = []

In [21]:
pdpf.ProfileReport(meta_df).to_file(doc_path+"/spiral-meta-profiling.html")


Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [5]:
# Creating a data structure with subjects time series and tremor levels
 
X = []
y = []

for path, level in zip(meta_df.abs_path, meta_df.level):
    df = pd.read_csv(path, sep="\s+",header=None,names=features,skiprows=1,usecols=[0,1,3,6])
    X.append(resample(df.values.astype('int16'), 4096))
    y.append(level)

In [6]:
with open('feat.npy', 'wb') as temp_feat:
    np.save(temp_feat, np.array(X, dtype=np.float32), allow_pickle=True)
with open('labels.npy', 'wb') as temp_labels:
    np.save(temp_labels, np.array(y, dtype=np.int8), allow_pickle=True)

In [7]:
with open('labels.npy', 'rb') as temp_labels:
    y_train = np.load(temp_labels, allow_pickle=True)
with open('feat.npy', 'rb') as temp_feat:
    X_train = np.load(temp_feat, allow_pickle=True)

In [8]:
X_train.shape

(53, 4096, 4)

In [9]:
y_train.shape

(53,)

In [10]:
dataset = tf.data.Dataset.from_tensor_slices((X_train, tf.one_hot(y_train, 3)))

In [11]:
tf.data.experimental.save(dataset, "./ethw.tfdata")

In [12]:
len(dataset)

53

In [13]:
for feat, label in tf.data.experimental.load("./ethw.tfdata").take(1):
    print(feat.shape)
    

(4096, 4)
